In [1]:
from sdRDM import DataModel
from sdRDM.generator import generate_python_api
import numpy as np
from datetime import datetime

In [2]:
# generate_python_api("specifications/new_calibration.md", "", "CaliPytion")

In [3]:
%reload_ext autoreload
%autoreload 2
from CaliPytion import Calibrator, Standard
from CaliPytion.modified import Sample

In [4]:
def random(xs):
    return xs+np.random.normal(0, 0.03, 1)


PARAM_A = 0.014

CONC = np.linspace(0, 200, 11)
ABSO = CONC*PARAM_A

In [5]:
standard = Standard(
    species_id="s0",
    name="ABTS",
    wavelength=340,
    ph=2,
    temperature=25,
    temperature_unit="C",
    created=datetime.now(),
)

# Add samples
for conc, abso in zip(CONC, ABSO):
    standard.add_to_samples(
        concentration=conc,
        conc_unit="umol / l",
        signal=random(abso),
    )
    standard.add_to_samples(
        concentration=conc,
        conc_unit="umol / l",
        signal=random(abso),
    )
    standard.add_to_samples(
        concentration=conc,
        conc_unit="umol / l",
        signal=random(abso),
    )

In [6]:
calibrator = Calibrator.from_standard(standard)

# Add models to calibrator
calibrator.add_to_models(name="linear", equation="a * concentration = signal")
calibrator.add_to_models(
    name="quadratic", equation="a * concentration**2 + b * concentration = signal")

print(calibrator)

Calibrator
├── id = calibrator0
├── standard
│   └── Standard
│       ├── id = standard1
│       ├── species_id = s0
│       ├── name = ABTS
│       ├── wavelength = 340.0
│       ├── samples
│       │   ├── 0
│       │   │   └── Sample
│       │   │       ├── id = sample0
│       │   │       ├── concentration = 0.0
│       │   │       ├── conc_unit = umol / l
│       │   │       └── signal = 0.02239851328463136
│       │   ├── 1
│       │   │   └── Sample
│       │   │       ├── id = sample1
│       │   │       ├── concentration = 0.0
│       │   │       ├── conc_unit = umol / l
│       │   │       └── signal = -0.02633775251814566
│       │   ├── 2
│       │   │   └── Sample
│       │   │       ├── id = sample2
│       │   │       ├── concentration = 0.0
│       │   │       ├── conc_unit = umol / l
│       │   │       └── signal = 0.013079283198162825
│       │   ├── 3
│       │   │   └── Sample
│       │   │       ├── id = sample3
│       │   │       ├── concentration = 20.0
│      

In [7]:
model = calibrator.models[1]
model

Model(id='model2', name='quadratic', equation='a * concentration**2 + b * concentration = signal', parameters=[], aic=None, bic=None, r2=None, residuals=[], rmsd=None)

In [8]:
res = model.fit_signals(
    concentrations=calibrator.concentrations,
    signals=calibrator.signals,
)

In [9]:
print(model)

Model
├── id = model2
├── name = quadratic
├── equation = a * concentration**2 + b * concentration = signal
├── parameters
│   ├── 0
│   │   └── Parameter
│   │       ├── id = parameter0
│   │       ├── name = a
│   │       ├── value = 3.4827077299165946e-07
│   │       ├── init_value = 0.1
│   │       ├── standard_error = 1.2794228308558355e-06
│   │       ├── lower_bound = -inf
│   │       └── upper_bound = inf
│   └── 1
│       └── Parameter
│           ├── id = parameter1
│           ├── name = b
│           ├── value = 0.013967066312244009
│           ├── init_value = 0.1
│           ├── standard_error = 0.00020756639271815187
│           ├── lower_bound = -inf
│           └── upper_bound = inf
├── aic = -219.19383633222947
├── bic = -216.2008212092965
├── r2 = 0.9985405622170008
├── residuals = [-0.02239851328463136, 0.02633775251814566, -0.013079283198162825, 0.02312953976017984, 0.0040671597158589146, ...]
└── rmsd = 0.033988780540944695



## concentration calculation

In [30]:
model.calculate_concentrations(
    signals=[1, 1.1, 2.21],
    cutoff=3,
    calibration_range=calibrator.concentrations
)

[71.4696302430382, 78.60263767221747, 157.60995132918583]

In [18]:
print(model)

Model
├── id = model2
├── name = quadratic
├── equation = a * concentration**2 + b * concentration = signal
├── parameters
│   ├── 0
│   │   └── Parameter
│   │       ├── id = parameter0
│   │       ├── name = a
│   │       ├── value = 3.4827077299165946e-07
│   │       ├── init_value = 0.1
│   │       ├── standard_error = 1.2794228308558355e-06
│   │       ├── lower_bound = -inf
│   │       └── upper_bound = inf
│   └── 1
│       └── Parameter
│           ├── id = parameter1
│           ├── name = b
│           ├── value = 0.013967066312244009
│           ├── init_value = 0.1
│           ├── standard_error = 0.00020756639271815187
│           ├── lower_bound = -inf
│           └── upper_bound = inf
├── aic = -219.19383633222947
├── bic = -216.2008212092965
├── r2 = 0.9985405622170008
├── residuals = [-0.02239851328463136, 0.02633775251814566, -0.013079283198162825, 0.02312953976017984, 0.0040671597158589146, ...]
└── rmsd = 0.033988780540944695

